# 한국어 개체명 인식 개요
이번 강의에서는 버트를 활용해서 한국어 개체명 인식을 다뤄보고자 합니다. 
![Imgur](https://i.imgur.com/PDdTLjy.png) 
데이터는 https://github.com/naver/nlp-challenge/ 에서 받아왔습니다.

개체명 추출 리더보드에서 제공되는 코퍼스는 문장에 나타난 개체명을 14개 분류 카테고리로 주석 작업이 되어있습니다.

![Imgur](https://i.imgur.com/it4uTE3.png)

문장을 입력하면 다음과 같은 형식으로 개체명이 분류됩니다.  

![Imgur](https://i.imgur.com/RpuZf6R.png)




# 목차
이번 실습은 <b>1) 네이버 개체명 인식 데이터 불러오기 및 전처리 2) BERT 인풋 만들기 3) 버트를 활용한 개체명 인식 모델 만들기 4) 훈련 및 성능 검증 5) 실제 데이터로 실습하기</b>로 구성되어 있습니다.

# BERT를 활용하여 한국어 개체명 인식기 만들기

## 개체명 인식 데이터 불러오기 및 전처리

개체명 인식을 위한 데이터를 다운 받습니다.

In [1]:
!wget https://github.com/naver/nlp-challenge/raw/master/missions/ner/data/train/train_data

--2022-06-25 05:28:59--  https://github.com/naver/nlp-challenge/raw/master/missions/ner/data/train/train_data
Resolving github.com (github.com)... 52.78.231.108
Connecting to github.com (github.com)|52.78.231.108|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/naver/nlp-challenge/master/missions/ner/data/train/train_data [following]
--2022-06-25 05:28:59--  https://raw.githubusercontent.com/naver/nlp-challenge/master/missions/ner/data/train/train_data
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16945023 (16M) [text/plain]
Saving to: ‘train_data’

train_data          100%[===================>]  16.16M  42.3MB/s    in 0.4s    

2022-06-25 05:29:02 (42.3 MB/s) - ‘train_data’ saved [16945023/16

분석에 필요한 모듈들을 임포트 합니다.

In [1]:
!pip install transformers
!pip install sacremoses
!pip install sentencepiece
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import re
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.8/dist-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/local/lib/python3.8/dist-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
/usr/local/lib/python3.8/dist-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/local/lib/python3.8/dist-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting 

/usr/local/lib/python3.8/dist-packages/pandas/compat/_optional.py:161: UserWarning: Pandas requires version '2.7.1' or newer of 'numexpr' (version '2.7.0' currently installed).
  warnings.warn(msg, UserWarning)


train 데이터를 불러 오겠습니다.

In [2]:
train = pd.read_csv("train_data", names=['src', 'tar'], sep="\t")
train = train.reset_index()
train

,index,src,tar
0,1,비토리오,PER_B
1,2,양일,DAT_B
2,3,만에,-
3,4,영사관,ORG_B
4,5,감호,CVL_B
...,...,...,...
769060,2,어째,-
769061,3,뭔가,-
769062,4,수상쩍은,-
769063,5,좌담,-


train 데이터에 마침표가 이상한 것들이 많아서 확실하게 .으로 수정해 주겠습니다.

In [3]:
train['src'] = train['src'].str.replace("．", ".", regex=False)

In [4]:
train.loc[train['src']=='.']

,index,src,tar
15,6,.,-
30,15,.,-
40,10,.,-
77,24,.,-
96,19,.,-
...,...,...,...
769013,11,.,-
769036,23,.,-
769053,17,.,-
769058,5,.,-


데이터를 전처리 해주겠습니다.  
한글, 영어, 소문자, 대문자, . 이외의 단어들을 모두 제거하겠습니다.

In [5]:
train['src'] = train['src'].astype(str)
train['tar'] = train['tar'].astype(str)

train['src'] = train['src'].str.replace(r'[^ㄱ-ㅣ가-힣0-9a-zA-Z.]+', "", regex=True)

데이터를 리스트 형식으로 변환합니다.

In [6]:
data = [list(x) for x in train[['index', 'src', 'tar']].to_numpy()]

데이터를 잘 보면 (인덱스, 단어, 개체) 로 이루어 진 것을 알 수 있습니다.  
인덱스가 1,2,3,4,5.. 이렇게 이어지다가 다시 1,2,3,4, 로 바뀌는데 숫자가 바뀌기 전까지가 한 문장을 의미합니다.

In [7]:
print(data[:20])

[[1, '비토리오', 'PER_B'], [2, '양일', 'DAT_B'], [3, '만에', '-'], [4, '영사관', 'ORG_B'], [5, '감호', 'CVL_B'], [6, '용퇴', '-'], [7, '항룡', '-'], [8, '압력설', '-'], [9, '의심만', '-'], [10, '가율', '-'], [1, '이', '-'], [2, '음경동맥의', '-'], [3, '직경이', '-'], [4, '8', 'NUM_B'], [5, '19mm입니다', 'NUM_B'], [6, '.', '-'], [1, '9세이브로', 'NUM_B'], [2, '구완', '-'], [3, '30위인', 'NUM_B'], [4, 'LG', 'ORG_B']]


라벨들을 추출하고, 딕셔너리 형식으로 저장하도록 하겠습니다.

In [8]:
label = train['tar'].unique().tolist()
label_dict = {word:i for i, word in enumerate(label)}
label_dict.update({"[PAD]":len(label_dict)})
index_to_ner = {i:j for j, i in label_dict.items()}

In [9]:
print(label_dict)

{'PER_B': 0, 'DAT_B': 1, '-': 2, 'ORG_B': 3, 'CVL_B': 4, 'NUM_B': 5, 'LOC_B': 6, 'EVT_B': 7, 'TRM_B': 8, 'TRM_I': 9, 'EVT_I': 10, 'PER_I': 11, 'CVL_I': 12, 'NUM_I': 13, 'TIM_B': 14, 'TIM_I': 15, 'ORG_I': 16, 'DAT_I': 17, 'ANM_B': 18, 'MAT_B': 19, 'MAT_I': 20, 'AFW_B': 21, 'FLD_B': 22, 'LOC_I': 23, 'AFW_I': 24, 'PLT_B': 25, 'FLD_I': 26, 'ANM_I': 27, 'PLT_I': 28, '[PAD]': 29}


In [10]:
print(index_to_ner)

{0: 'PER_B', 1: 'DAT_B', 2: '-', 3: 'ORG_B', 4: 'CVL_B', 5: 'NUM_B', 6: 'LOC_B', 7: 'EVT_B', 8: 'TRM_B', 9: 'TRM_I', 10: 'EVT_I', 11: 'PER_I', 12: 'CVL_I', 13: 'NUM_I', 14: 'TIM_B', 15: 'TIM_I', 16: 'ORG_I', 17: 'DAT_I', 18: 'ANM_B', 19: 'MAT_B', 20: 'MAT_I', 21: 'AFW_B', 22: 'FLD_B', 23: 'LOC_I', 24: 'AFW_I', 25: 'PLT_B', 26: 'FLD_I', 27: 'ANM_I', 28: 'PLT_I', 29: '[PAD]'}


데이터를 문장들과 개체들로 분리합니다.  
tups[0], tups[1],... 에 각각의 문장에 해당하는 단어와 개체 번호가 저장이 되게 됩니다.

In [11]:
tups = []
temp_tup = []
temp_tup.append(data[0][1:])
sentences = []
targets = []
for i, j, k in data:
  
  if i != 1:
    temp_tup.append([j,label_dict[k]])
  if i == 1:
    if len(temp_tup) != 0:
      tups.append(temp_tup)
      temp_tup = []
      temp_tup.append([j,label_dict[k]])

tups.pop(0)

[['비토리오', 'PER_B']]

In [12]:
print(tups[0], tups[1])

[['비토리오', 0], ['양일', 1], ['만에', 2], ['영사관', 3], ['감호', 4], ['용퇴', 2], ['항룡', 2], ['압력설', 2], ['의심만', 2], ['가율', 2]] [['이', 2], ['음경동맥의', 2], ['직경이', 2], ['8', 5], ['19mm입니다', 5], ['.', 2]]


tups를 보면 [(단어, 개체), (단어, 개체), (단어, 개체)]의 형식으로 저장이 되어 있는데, 이거를 (단어, 단어, 단어, 단어), (개체, 개체, 개체, 개체) 형식으로 변환하도록 하겠습니다.


In [13]:
sentences = []
targets = []
for tup in tups:
  sentence = []
  target = []
  sentence.append("[CLS]")
  target.append(label_dict['-'])
  for i, j in tup:
    sentence.append(i)
    target.append(j)
  sentence.append("[SEP]")
  target.append(label_dict['-'])
  sentences.append(sentence)
  targets.append(target)

In [14]:
sentences[0]

['[CLS]',
 '비토리오',
 '양일',
 '만에',
 '영사관',
 '감호',
 '용퇴',
 '항룡',
 '압력설',
 '의심만',
 '가율',
 '[SEP]']

In [15]:
targets[0]

[2, 0, 1, 2, 3, 4, 2, 2, 2, 2, 2, 2]

## 버트 인풋 만들기

구글의 multilinguial-bert를 활용하도록 하겠습니다.

In [16]:
tokenizer = BertTokenizer.from_pretrained('jhgan/ko-sroberta-multitask')
# bert-base-multilingual-cased
# monologg/koelectra-base-v3-discriminator
# monologg/kobigbird-bert-base
# monologg/koelectra-base-v3-naver-ner
# monologg/koelectra-v3-klue-sts
# klue/bert-base
# klue/roberta-base
# jhgan/ko-sroberta-multitask

https://huggingface.co/jhgan/ko-sroberta-multitask/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /home/work/.cache/huggingface/transformers/tmpuarvmoaa


Downloading:   0%|          | 0.00/585 [00:00<?, ?B/s]

storing https://huggingface.co/jhgan/ko-sroberta-multitask/resolve/main/tokenizer_config.json in cache at /home/work/.cache/huggingface/transformers/ce6b286c855114af5f83557f305decac64eecd63edff79558124ba2819d7b8df.d0f75856a5c8279dc23d3dfd032445ff332e213952c2a14775b1560d527c1ad3
creating metadata file for /home/work/.cache/huggingface/transformers/ce6b286c855114af5f83557f305decac64eecd63edff79558124ba2819d7b8df.d0f75856a5c8279dc23d3dfd032445ff332e213952c2a14775b1560d527c1ad3
https://huggingface.co/jhgan/ko-sroberta-multitask/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /home/work/.cache/huggingface/transformers/tmpp8lhx5mu


Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

storing https://huggingface.co/jhgan/ko-sroberta-multitask/resolve/main/vocab.txt in cache at /home/work/.cache/huggingface/transformers/fc6f857ebbf07f6038dbc22d56f6ecc837598c490ff18cf28f185ac42da2c960.d1b86bed49516351c7bb29b19d7e7be2ab53b931bcb1f9b2aacfb71f2124d25a
creating metadata file for /home/work/.cache/huggingface/transformers/fc6f857ebbf07f6038dbc22d56f6ecc837598c490ff18cf28f185ac42da2c960.d1b86bed49516351c7bb29b19d7e7be2ab53b931bcb1f9b2aacfb71f2124d25a
https://huggingface.co/jhgan/ko-sroberta-multitask/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /home/work/.cache/huggingface/transformers/tmpmz_ch00d


Downloading:   0%|          | 0.00/156 [00:00<?, ?B/s]

storing https://huggingface.co/jhgan/ko-sroberta-multitask/resolve/main/special_tokens_map.json in cache at /home/work/.cache/huggingface/transformers/dd6a66737e693d1fe4d593341deffa1fa2611b9c69ef882ea2895ba79d3ecfa7.f886166424e457f0fc75f92e81205faabe843b2dbbbef6b25f9d8ec69f64bc7d
creating metadata file for /home/work/.cache/huggingface/transformers/dd6a66737e693d1fe4d593341deffa1fa2611b9c69ef882ea2895ba79d3ecfa7.f886166424e457f0fc75f92e81205faabe843b2dbbbef6b25f9d8ec69f64bc7d
https://huggingface.co/jhgan/ko-sroberta-multitask/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /home/work/.cache/huggingface/transformers/tmp0o_xnlq2


Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

storing https://huggingface.co/jhgan/ko-sroberta-multitask/resolve/main/tokenizer.json in cache at /home/work/.cache/huggingface/transformers/897a475d9a163b55da6538df899e60740add9aff66d917152f7b0bd84b7c61d7.cd6b94e2188d9d45cc77ccb5847447b918ab52e4a41e88c84fce5b3cfc2b8088
creating metadata file for /home/work/.cache/huggingface/transformers/897a475d9a163b55da6538df899e60740add9aff66d917152f7b0bd84b7c61d7.cd6b94e2188d9d45cc77ccb5847447b918ab52e4a41e88c84fce5b3cfc2b8088
loading file https://huggingface.co/jhgan/ko-sroberta-multitask/resolve/main/vocab.txt from cache at /home/work/.cache/huggingface/transformers/fc6f857ebbf07f6038dbc22d56f6ecc837598c490ff18cf28f185ac42da2c960.d1b86bed49516351c7bb29b19d7e7be2ab53b931bcb1f9b2aacfb71f2124d25a
loading file https://huggingface.co/jhgan/ko-sroberta-multitask/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/jhgan/ko-sroberta-multitask/resolve/main/special_tokens_map.json from cache at /home/work/.cache/hugging

In [17]:
tokenizer.tokenize("대한민국 만세.")

['대한민국', '만세', '.']

여기서부터가 중요한데, 문장을 토크나이징 하고 개체(target)을 토크나이징 한 문장에 맞추도록 하겠습니다.  
문장 "대한민국 만세." 는 사실 (대한민국, 개체1), (만세., 개체2) 을 가지고 있는데 토크나이징을 하면 '▁대한민국', '▁만', '세', '.' 로 토크나이징이 됩니다.  
여기서 그렇다면 ( ▁대한민국, 개체1) , (▁만, 개체2), (세, 개체2), (., 개체 2) 와 같은 방식으로 각 개체를 부여해주어야 합니다.

In [18]:
def tokenize_and_preserve_labels(sentence, text_labels):
  tokenized_sentence = []
  labels = []

  for word, label in zip(sentence, text_labels):

    tokenized_word = tokenizer.tokenize(word)
    n_subwords = len(tokenized_word)

    tokenized_sentence.extend(tokenized_word)
    labels.extend([label] * n_subwords)

  return tokenized_sentence, labels

In [19]:
tokenized_texts_and_labels = [tokenize_and_preserve_labels(sent, labs) for sent, labs in zip(sentences, targets)]

In [20]:
print(tokenized_texts_and_labels[:2])
# [(문장, 개체들), (문장, 개체들),...] 형식으로 저장되어 있음.

[(['[CLS]', '비', '##토리', '##오', '양일', '만', '##에', '영사관', '감', '##호', '용', '##퇴', '항', '##룡', '압력', '##설', '의심', '##만', '가', '##율', '[SEP]'], [2, 0, 0, 0, 1, 2, 2, 3, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]), (['[CLS]', '이', '음', '##경', '##동맥', '##의', '직경', '##이', '8', '19', '##m', '##m', '##입니다', '.', '[SEP]'], [2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5, 5, 2, 2])]


(문장, 개체들), (문장, 개체들) 을 [문장, 문장, 문장, ...] , [개체들, 개체들 개체들,,,,]로 분리해주도록 하겠습니다.

In [21]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [22]:
tokenized_texts[1]

['[CLS]',
 '이',
 '음',
 '##경',
 '##동맥',
 '##의',
 '직경',
 '##이',
 '8',
 '19',
 '##m',
 '##m',
 '##입니다',
 '.',
 '[SEP]']

In [23]:
labels[1]

[2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5, 5, 2, 2]

문장의 길이가 상위 2.5%(88) 인 지점을 기준으로 문장의 길이를 정하도록 하겠습니다.  
만약 문장의 길이가 88보다 크면 문장이 잘리게 되고, 길이가 88보다 작다면 패딩이 되어 모든 문장의 길이가 88로 정해지게 됩니다.

In [25]:
print(np.quantile(np.array([len(x) for x in tokenized_texts]), 0.975))
max_len = 80
bs = 32

80.0


버트에 인풋으로 들어갈 train 데이터를 만들도록 하겠습니다.  
버트 인풋으로는   
input_ids : 문장이 토크나이즈 된 것이 숫자로 바뀐 것,   
attention_masks : 문장이 토크나이즈 된 것 중에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 마스킹  
[input_ids, attention_masks]가 인풋으로 들어갑니다.

In [26]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype="int", value=tokenizer.convert_tokens_to_ids("[PAD]"), truncating="post", padding="post")

In [27]:
input_ids[1]

array([    0,  1504,  1500,  2382, 28726,  2079, 27450,  2052,    28,
        3630,  2037,  2037, 12190,    18,     2,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1])

정답에 해당하는 개체들을 만들어 보겠습니다.  
패딩에 해당하는 부분은 label_dict([PAD])(29)가 들어가게 되겠습니다.

In [28]:
tags = pad_sequences([lab for lab in labels], maxlen=max_len, value=label_dict["[PAD]"], padding='post',\
                     dtype='int', truncating='post')

In [29]:
tags[1]

array([ 2,  2,  2,  2,  2,  2,  2,  2,  5,  5,  5,  5,  5,  2,  2, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29])

어텐션 마스크를 만들어 주겠습니다.

In [30]:
attention_masks = np.array([[int(i != tokenizer.convert_tokens_to_ids("[PAD]")) for i in ii] for ii in input_ids])

train 데이터에서 10% 만큼을 validation 데이터로 분리해 주겠습니다.

In [31]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,
                                                            random_state=20172848, test_size=0.2)

In [32]:
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=20172848, test_size=0.2)

## 개체명 인식 모델 만들기

In [32]:
# TPU 작동을 위해 실행
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)

In [33]:
SEQ_LEN = max_len
def create_model():
  model = TFBertModel.from_pretrained("jhgan/ko-sroberta-multitask", from_pt=True, num_labels=len(label_dict), output_attentions=False,
    output_hidden_states=False)

  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids') # 토큰 인풋
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks') # 마스크 인풋

  bert_outputs = model([token_inputs, mask_inputs])
  bert_outputs = bert_outputs[0] # shape : (Batch_size, max_len, 30(개체의 총 개수))
  nr = tf.keras.layers.Dense(30, activation='softmax')(bert_outputs) # shape : (Batch_size, max_len, 30)
  
  nr_model = tf.keras.Model([token_inputs, mask_inputs], nr)
  
  nr_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00002), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
      metrics=['sparse_categorical_accuracy'])
  nr_model.summary()
  return nr_model

## 훈련 및 성능 검증

In [34]:
# strategy = tf.distribute.experimental.TPUStrategy(resolver)
# # TPU를 활용하기 위해 context로 묶어주기
# with strategy.scope():
#   nr_model = create_model()
#   nr_model.fit([tr_inputs, tr_masks], tr_tags, validation_data=([val_inputs, val_masks], val_tags), epochs=3, shuffle=False, batch_size=bs)

In [35]:
# 만약 TPU를 사용하지 않고 GPU를 사용한다면
nr_model = create_model()
nr_model.fit([tr_inputs, tr_masks], tr_tags, validation_data=([val_inputs, val_masks], val_tags), epochs=3, shuffle=False, batch_size=bs)

https://huggingface.co/jhgan/ko-sroberta-multitask/resolve/main/config.json not found in cache or force_download set to True, downloading to /home/work/.cache/huggingface/transformers/tmplue8z_zp


Downloading:   0%|          | 0.00/744 [00:00<?, ?B/s]

storing https://huggingface.co/jhgan/ko-sroberta-multitask/resolve/main/config.json in cache at /home/work/.cache/huggingface/transformers/da3a48f12ce6c8a4a197bf4b9973e15c918cd365af12e54cbdee6e3ef7e14a8b.47d0108a14142aa67a7bba92ec6208754dca219b63b124ad3b7db7d88fa72054
creating metadata file for /home/work/.cache/huggingface/transformers/da3a48f12ce6c8a4a197bf4b9973e15c918cd365af12e54cbdee6e3ef7e14a8b.47d0108a14142aa67a7bba92ec6208754dca219b63b124ad3b7db7d88fa72054
loading configuration file https://huggingface.co/jhgan/ko-sroberta-multitask/resolve/main/config.json from cache at /home/work/.cache/huggingface/transformers/da3a48f12ce6c8a4a197bf4b9973e15c918cd365af12e54cbdee6e3ef7e14a8b.47d0108a14142aa67a7bba92ec6208754dca219b63b124ad3b7db7d88fa72054
You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Model config BertConfig {
  "_name_or_path": "klue/roberta-base",
  "architectures": [
  

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

storing https://huggingface.co/jhgan/ko-sroberta-multitask/resolve/main/pytorch_model.bin in cache at /home/work/.cache/huggingface/transformers/22e78be8f4b1a6ff8df939ab8c2851184d8112721ff819c97ac07f58ad968f7a.8f25dfefa65205551910bb5bc1592ba74979251e8e922a6e4c028f71992630a9
creating metadata file for /home/work/.cache/huggingface/transformers/22e78be8f4b1a6ff8df939ab8c2851184d8112721ff819c97ac07f58ad968f7a.8f25dfefa65205551910bb5bc1592ba74979251e8e922a6e4c028f71992630a9
loading weights file https://huggingface.co/jhgan/ko-sroberta-multitask/resolve/main/pytorch_model.bin from cache at /home/work/.cache/huggingface/transformers/22e78be8f4b1a6ff8df939ab8c2851184d8112721ff819c97ac07f58ad968f7a.8f25dfefa65205551910bb5bc1592ba74979251e8e922a6e4c028f71992630a9
Loading PyTorch weights from /home/work/.cache/huggingface/transformers/22e78be8f4b1a6ff8df939ab8c2851184d8112721ff819c97ac07f58ad968f7a.8f25dfefa65205551910bb5bc1592ba74979251e8e922a6e4c028f71992630a9
PyTorch checkpoint contains 110,6

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 80)]         0           []                               
                                                                                                  
 input_masks (InputLayer)       [(None, 80)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  110618112   ['input_word_ids[0][0]',         
                                thPoolingAndCrossAt               'input_masks[0][0]']            
                                tentions(last_hidde                                               
                                n_state=(None, 80,                                            

/usr/local/lib/python3.8/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1638/1638 [==============================] - 216s 128ms/step - loss: 0.1936 - sparse_categorical_accuracy: 0.9444 - val_loss: 0.1272 - val_sparse_categorical_accuracy: 0.9596
Epoch 2/3
1638/1638 [==============================] - 209s 127ms/step - loss: 0.1160 - sparse_categorical_accuracy: 0.9629 - val_loss: 0.1172 - val_sparse_categorical_accuracy: 0.9626
Epoch 3/3
1638/1638 [==============================] - 208s 127ms/step - loss: 0.0917 - sparse_categorical_accuracy: 0.9696 - val_loss: 0.1162 - val_sparse_categorical_accuracy: 0.9638


In [36]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [37]:
y_predicted = nr_model.predict([val_inputs, val_masks])

In [38]:
f_label = [i for i, j in label_dict.items()]
val_tags_l = [index_to_ner[x] for x in np.ravel(val_tags).astype(int).tolist()]
y_predicted_l = [index_to_ner[x] for x in np.ravel(np.argmax(y_predicted, axis=2)).astype(int).tolist()]
f_label.remove("[PAD]")

각 개체별 f1 score를 측정하도록 하겠습니다.  
참고로 micro avg는 전체 정답을 기준으로 f1 score을 측정한 것이며,  
macro avg는 각 개체별 f1 score를 가중평균 한 것입니다.

In [39]:
print(classification_report(val_tags_l, y_predicted_l, labels=f_label))

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1

              precision    recall  f1-score   support

       PER_B       0.80      0.87      0.83     18983
       DAT_B       0.86      0.86      0.86      8542
           -       0.94      0.95      0.95    237060
       ORG_B       0.89      0.82      0.85     21077
       CVL_B       0.85      0.79      0.82     25685
       NUM_B       0.93      0.93      0.93     22643
       LOC_B       0.81      0.77      0.79      8415
       EVT_B       0.81      0.78      0.79      5392
       TRM_B       0.84      0.78      0.81     10749
       TRM_I       0.47      0.65      0.54      1159
       EVT_I       0.73      0.87      0.80      2775
       PER_I       0.74      0.82      0.78      2669
       CVL_I       0.54      0.46      0.50      1303
       NUM_I       0.67      0.84      0.74      2645
       TIM_B       0.81      0.83      0.82      1002
       TIM_I       0.85      0.95      0.90       456
       ORG_I       0.77      0.70      0.73      2188
       DAT_I       0.79    

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 실제 데이터로 실습하기

In [40]:
def ner_inference(test_sentence):
  

  tokenized_sentence = np.array([tokenizer.encode(test_sentence, max_length=max_len, truncation=True, padding='max_length')])
  tokenized_mask = np.array([[int(x!=1) for x in tokenized_sentence[0].tolist()]])
  ans = nr_model.predict([tokenized_sentence, tokenized_mask])
  ans = np.argmax(ans, axis=2)

  tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence[0])
  new_tokens, new_labels = [], []
  for token, label_idx in zip(tokens, ans[0]):
    if (token.startswith("##")):
      new_labels.append(index_to_ner[label_idx])
      new_tokens.append(token[2:])
    elif (token=='[CLS]'):
      pass
    elif (token=='[SEP]'):
      pass
    elif (token=='[PAD]'):
      pass
    elif (token != '[CLS]' or token != '[SEP]'):
      new_tokens.append(token)
      new_labels.append(index_to_ner[label_idx])

  for token, label in zip(new_tokens, new_labels):
      print("{}\t{}".format(label, token))

In [41]:
ner_inference("문재인 대통령은 1953년 1월 24일 경상남도 거제시에서 아버지 문용형과 어머니 강한옥 사이에서 둘째(장남)로 태어났다.")

PER_B	문재인
-	대통령
CVL_B	은
DAT_B	1953
DAT_B	년
DAT_I	1
DAT_I	월
DAT_I	24
DAT_I	일
LOC_B	경상남도
LOC_B	거제시
LOC_B	에서
CVL_B	아버지
PER_B	문
PER_B	용
PER_B	형
PER_B	과
CVL_B	어머니
PER_B	강한
PER_B	옥
-	사이
-	에서
NUM_B	둘째
CVL_B	(
CVL_B	장남
-	)
-	로
-	태어났
-	다
-	.


In [42]:
ner_inference("9세이브로 구완 30위인 LG 박찬형은 평균자책점이 16.45로 준수한 편이지만 22이닝 동안 피홈런이 31개나 된다.")

NUM_B	9
NUM_B	세이
NUM_B	브
NUM_B	로
-	구
-	완
NUM_B	30
NUM_B	위
NUM_B	인
ORG_B	LG
PER_B	박찬
PER_B	형
PER_B	은
-	평균
-	자
-	책
-	점
-	이
NUM_B	16
NUM_B	.
NUM_B	45
NUM_B	로
-	준수
-	한
-	편
-	이지
-	만
NUM_B	22
NUM_B	이
NUM_B	닝
-	동안
-	피
-	홈
-	런
-	이
NUM_B	31
NUM_B	개
NUM_B	나
-	된다
-	.


In [43]:
ner_inference("인공지능의 역사는 20세기 초반에서 더 거슬러 올라가보면 이미 17~18세기부터 태동하고 있었지만 이때는 인공지능 그 자체보다는 뇌와 마음의 관계에 관한 철학적인 논쟁 수준에 머무르고 있었다. 그럴 수 밖에 없는 것이 당시에는 인간의 뇌 말고는 정보처리기계가 존재하지 않았기 때문이다. ")

TRM_B	인공지능
-	의
-	역사
-	는
NUM_B	20
NUM_B	세기
-	초반
-	에서
-	더
-	거슬러
-	올라가
-	보
-	면
-	이미
NUM_B	17
NUM_B	~
NUM_B	18
NUM_B	세기
NUM_B	부터
-	태동
-	하고
-	있
-	었
-	지만
-	이때
-	는
TRM_B	인공지능
-	그
-	자체
-	보
-	다는
-	뇌
-	와
-	마음
-	의
-	관계
-	에
-	관한
-	철학
-	적인
-	논쟁
-	수준
-	에
-	머무르
-	고
-	있
-	었
-	다
-	.
-	그럴
-	수
-	밖에
-	없
-	는
-	것
-	이
-	당시
-	에
-	는
-	인간
-	의
-	뇌
-	말
-	고
-	는
-	정보
-	처리
-	기계
-	가
-	존재
-	하
-	지
-	않
-	았
-	기
-	때문
-	이다
-	.


In [44]:
ner_inference("안녕하세요. 저는 이지평이라고 합니다. 국민대학교에 다니고 있으며, 현재는 4학년 1학기를 마치고 SKT AI Fellowship 4기에 참여 중입니다.")

-	안녕
-	하
-	세요
-	.
-	저
-	는
PER_B	이지
PER_B	평
PER_B	이
PER_B	라고
-	합니다
-	.
ORG_B	국민대
ORG_B	학교
ORG_B	에
-	다니
-	고
-	있
-	으며
-	,
-	현재
-	는
NUM_B	4
NUM_B	학년
NUM_I	1
NUM_I	학
NUM_I	기
NUM_I	를
-	마치
-	고
ORG_B	SKT
EVT_I	AI
EVT_I	Fe
EVT_I	ll
EVT_I	ow
EVT_I	ship
NUM_B	4
NUM_B	기에
-	참여
-	중
-	입니다
-	.


In [45]:
ner_inference("짱구는 못말려")

-	짱
-	구
-	는
-	못
-	말
-	려
